## Zadanie domowe: morfologiczna gra w życie – John Conway

### Wykorzystanie operacji LUT w przekształceniu trafi, nie trafi
  - Szybszą metodą wykonania transformacji trafi, nie trafi może być operacja LUT.
  - Technika polega na zakodowaniu wyników wszystkich interesujących  konfiguracji, a następnie podczas przetwarzania wykorzystania operacji LUT.
  - Dla otoczenia 3x3 możliwe jest 512 różnych konfiguracji.
  - Aby praktycznie zrealizować operacje, każdej konfiguracji należy nadać unikalny indeks. Jedną z możliwości jest pomnożenie elementu strukturalnego przez macierz (mnożenie odpowiednich elementów):
  ```
  [[1, 8,  64],
   [ 2, 16, 128],
   [ 4, 32, 256]]
  ```
  Przykładowo elementowi:
  ```
  [[1, 1, 0],
   [ 1, 0, 1],
   [ 1, 0, 1]]
  ```
  odpowiada liczba: 1(1) + 2(1) + 4(1) + 8(1) + 128(1) + 256(1) = 399.
  
### Przykład działania metody – detekcja punktów końcowych na obrazie.
  - założenie: punkt końcowy to punkt, który ma dokładnie jednego sąsiada,
  - zdefiniuj funkcję, która jako argument pobiera otoczenie, a zwraca 0 lub 1 w zależności od tego, czy rozpatrywany punkt jest końcowy np. dla sąsiedztwa 3×3 punkt będzie końcowy, jeżeli jest zapalony i ma tylko jednego sąsiada (czyli suma pikseli jest równa 2).
  - wygeneruj przekodowanie LUT.
  - wczytaj obraz szkielet.bmp (należy go przekształcić, aby uzyskać dwuwymiarową tablicę o wartościach 0-1). Wykorzystując wygenerowane przekodowanie LUT wykonaj detekcję zakończeń. Wyświetl obraz oryginalny i po przekodowaniu LUT.

### Gra w życie

Reguły gry w życie:
  - każdy piksel biały, który ma dwóch lub trzech sąsiadów (białych) przeżywa,
  - każdy piksel biały, który ma 0,1 lub więcej niż trzech sąsiadów (białych) nie przeżywa (głód lub przeludnienie),
  - jeżeli jakieś pole (czarne) sąsiaduje dokładnie z trzema pikselami białymi, to na tym polu ,,rodzi'' się nowy piksel biały.

Zadanie:
  - za pomocą mechanizmu LUT (opisanego wcześniej) należy zaimplementować morfologiczną gre w życie,
  - najważniejszym elementem jest funkcja opisująca reguły gry,
  - symulacje należny przeprowadzić dla plansz dostarczonych w pliku gra.py,
  - dobrze jest wykonać kilka iteracji – zobaczyć jak zmienia się kształt,
  - inne ciekawe kształty do znalezienia w internecie.


In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

if not os.path.exists("gra.py") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/10_Morphology/gra.py --no-check-certificate

def apply_lut(image, lut):
    padded = np.pad(image, pad_width=1, mode='constant')
    output = np.zeros_like(image)

    weights = np.array([[1, 8, 64],
                        [2, 16, 128],
                        [4, 32, 256]])

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            window = padded[i:i+3, j:j+3]
            idx = np.sum(window * weights)
            output[i, j] = lut[idx]
    return output

def game_of_life_rule(neighborhood):
    center = neighborhood[1, 1]
    neighbors = np.sum(neighborhood) - center
    if center == 1:
        return 1 if neighbors in (2, 3) else 0
    else:
        return 1 if neighbors == 3 else 0

def generate_life_lut():
    lut = np.zeros(512, dtype=np.uint8)
    for i in range(512):
        binary = [(i >> bit) & 1 for bit in range(9)]
        neighborhood = np.array(binary, dtype=np.uint8).reshape((3, 3))
        lut[i] = game_of_life_rule(neighborhood)
    return lut

def simulate_life(binary, iterations=1):
    lut = generate_life_lut()
    current = binary.copy()
    for _ in range(iterations):
        current = apply_lut(current, lut)
    return current

In [ ]:
from gra import *

fig, ax = plt.subplots()
ax.imshow(plansza1, cmap='gray')
ax.axis("off")

for i in range(8):
    plansza1 = simulate_life(plansza1, 1)

    fig, ax = plt.subplots()
    ax.imshow(plansza1, cmap='gray')
    ax.axis("off")

In [ ]:
from gra import *

fig, ax = plt.subplots()
ax.imshow(plansza2, cmap='gray')
ax.axis("off")

for _ in range(7):
    fig, ax = plt.subplots(1, 5)
    for i in range(5):
        plansza2 = simulate_life(plansza2, 1)

        ax[i].imshow(plansza2, cmap='gray')
        ax[i].axis("off")